<a href="https://colab.research.google.com/github/yichin17/LLM/blob/main/16_gensim_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2

--2024-01-05 04:45:44--  https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230609687 (220M) [application/octet-stream]
Saving to: ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20231201-pag 100%[===================>] 219.93M  3.99MB/s    in 57s     

2024-01-05 04:46:41 (3.87 MB/s) - ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’ saved [230609687/230609687]



In [2]:
!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 6.5 MB/s eta 0:00:00


In [3]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})


wiki_corpus


next(iter(wiki_corpus.get_texts()))[:10]

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

In [4]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

7344 articles processed.


In [8]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

KeyboardInterrupt: 

In [ ]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')